<a href="https://colab.research.google.com/github/martians-sheep/pl_task_recomended_csd/blob/main/elastic_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elasticsearch で類似検索を実行するサンプル
## 前提条件
このサンプルコードではElasticsearchはv8.13.0を使用します。
またElasticsearchを外部に作成し、そこへアクセスする形となります。

In [ ]:
# Elasticsearch ライブラリのインストール
!pip install elasticsearch
from elasticsearch import Elasticsearch

In [12]:
# Elasticsearchのエンドポイントとポート番号を指定
es_endpoint = "{Elasticsearchのエンドポイント}"
username = "{username} ex. elasticなど"
password = "{password}"

In [ ]:
# Elasticsearchクライアントの作成
try:

    # Elasticsearchクライアントの作成
    es = Elasticsearch(
        hosts=[es_endpoint],
        http_auth=(username,password)
    )

    # Elasticsearchへの接続を確認
    if es.ping():
        print("Elastic Searchへの接続が確認されました。")
    else:
        print("Elastic Searchへの接続に失敗しました。")

except Exception as e:
    print(f"Elastic Searchへの接続中にエラーが発生しました: {e}")

In [ ]:
# Elasticsearchの情報表示
print(es.info())

In [ ]:
# Numpyのインストール
!pip install numpy

In [21]:
# 大量のドキュメントをバイナリ化したNumPyバイナリファイルからデータを読み込む
import numpy as np
vectors = np.load("./embeddings_all.npy")

# インデックス登録

In [26]:
index_name = "my_vector_index"
mapping = {
    "mappings": {
        "properties": {
            "my_vector": {
                "type": "dense_vector",
                "dims": vectors.shape[1]
            }
        }
    }
}

In [ ]:
# インデックス作成
es.indices.create(index=index_name, body=mapping)

In [28]:
# ベクトルデータをElasticsearchにインデックス化
for i, vector in enumerate(vectors):
    doc = {
        "my_vector": vector.tolist()
    }
    es.index(index=index_name, body=doc)

In [24]:
# インデックスの削除関数
def delete_index(es, index_name):
    """
    Elasticsearchのインデックスを削除する関数

    :param es: Elasticsearchクライアントオブジェクト
    :param index_name: 削除するインデックスの名前
    """
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    else:
        print(f"Index '{index_name}' does not exist.")

In [ ]:
# インデックスの削除
# delete_index(es, index_name)

In [ ]:
# 検索対象のテキストのベクトル化(OpenAI の Embeddingを利用)
!pip install openai

In [34]:
from openai import OpenAI
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# クライアントの準備
client = OpenAI()

In [39]:
# テキストを読み込む
txt_file_path = "./法律AIの活用に関する研究報告書.txt"

with open(txt_file_path, "r", encoding="utf-8") as file:
    in_text = file.read()

In [40]:
# 作業対象のファイルをベクトル化する
response =client.embeddings.create(
    input=in_text,
    model="text-embedding-ada-002"
)

# ベクトル化したデータをnumpy配列に変換
in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype("float32")

In [49]:
# コサイン類似度のクエリ(上手く実行できていない)
query = {
    "query": {
        "function_score": {
            "query": {"match_all": {}},
            "script_score": {
                "script": {
                    "source": "cosineSimilarity(params.query_vector, doc['my_vector'].vectorValue())",
                    "params": {"query_vector": in_embeds.tolist()}
                }
            }
        }
    }
}

In [68]:
# knn(k近傍)アルゴリズムでクエリの構築
query = {
    "query": {
        "knn": {
            "field": "my_vector",
            "query_vector": in_embeds.flatten().tolist(),
            "num_candidates": 100
        }
    },
      "size": 5
}

In [ ]:
!# 検索結果を取得
search_result = es.search(index=index_name, body=query)
# 検索結果を表示
print("Search results:")
for hit in search_result["hits"]["hits"]:
    print("doc Id",hit["_id"])
    print("Score:", hit["_score"])
    print("Vector:", hit["_source"]["my_vector"])
    print("---")